# New Developments in Deep Learning: Isotropic Graph Neural Networks
*Phuc Tran Truong (558919), Maike Basmer (576844)*


## Introduction
Graphs are used to model and analyze a variety of phenomena – be it social networks, protein-protein interactions or citation networks. These are rather irregular in comparison to regular structures like sequences or grids, which can be represented as a graph as well. Popular Machine Learning models have been tailored to these regular forms of graphs, specifically Recurrent Neural Networks (RNNs) (Rumelhart et al., 1986a) for sequences (e.g. sentences, time series) and Convolutional Neural Networks (CNNs) (LeCun, 1989) for grids (e.g. images). To be able to deal with general graphs however, Graph Neural Networks (GNNs) were devised (Hamilton, 2020, p. 47-48). GNNs base upon a differentiable message passing scheme, i.e. vector messages are passed between nodes and updated using neural networks (Hamilton, 2020, p. 48). In this project, the focus is on Graph Convolutional Networks (GCNs) (Kipf & Welling, 2016), but before going into detail, we will first define some important terms and illustrate potential use cases as well as previous approaches.

## Background 

A graph is commonly defined as a tuple $G = (V, E)$ where $V$ denotes the set of vertices and $E \subseteq V \times V$ is the set of edges between vertices. The edge relation is typically represented as an adjacency matrix $M$ where $M_{i,j} = 1$ iff $(i,j) \in E$. One or more features may be assigned to the vertices and/or edges in a graph. Nodes may further be encoded in the form of embeddings, i.e., as "low-dimensional vectors that summarize their graph position and the structure in their local neighborhood" (Hamilton, 2020, p. 29). Hence, node features describe the properties of the respective node in the given graph, thus are given explicitly, whereas a node embedding represents a node in a latent space.

Common tasks related to graphs are node classification, link prediction, graph classification and community detection (Hamilton, 2020, p. 4-8). In this seminar project, we will focus on node classification using Relational Graph Convolutional Networks (R-GCN). Node classification refers to predicting the labels for a set of unlabelled nodes in a graph, given a (usually rather small) set of labelled nodes of the same graph (Hamilton, 2020, p.5). \\
Originally, this was achieved by assigning some kind of node-level statistics like the node degree to each node and using these as input features to a standard machine learning classifier (Hamilton, 2020, p. 9-13). Another approach consists in *encoding* nodes as so-classed shallow embeddings, which are computed for each node uniquely, and then reconstructing graph characteristics, like an edge between two nodes, from these embeddings (corresponding to a *decoding* step, e.g. based on random walks)  (Hamilton, 2020, p. 29-35). \\
However, shallow embeddings have some major shortcomings  (Hamilton, 2020, p. 36-37): they do not allow for parameter sharing, which would improve the computational efficiency and constitute a form of regularization. They further do not incorporate the graph structure or feature information, which could improve the quality of the embeddings. Lastly, they cannot generalize to nodes unseen during the training. Graph Neural Networks (GNNs) tackle these deficiencies, and thus enable the computation of more sophisticated embeddings that can be used to classify nodes or predict links (Hamilton, 2020, p. 37). In the following, we are going to elaborate on (Relational) Graph Convolutional Networks in order to understand their applicability to the problem of node classification.

## Graph Convolutional Networks (GCNs)
Kipf & Welling (2016) proposed Graph Convolutional Networks (GCNs) that make use of a first-order approximation of spectral graph convolutions. In essence, that corresponds to the aforementioned message passing mechanism enhanced with symmetric normalization, i.e. the representation of a node $i$ is computed by aggregating and normalizing the information held by its neighbors $N_i$ and combining the result with $i$'s previous representation. Accordingly, a GCN layer consists of normalized message passing operations, followed by a non-linear activation. This idea is condensed in the following formula which represents the update of node $i$ in layer $l+1$:
> $h_i^{l+1} = \sigma\left(\sum_{j\in N_i}{c_{i,j}}W_i^{(l)^T}h_j^{(l)}+\underbrace{W_0^{(l)^T}h_i^{(l)}}_{\text{self-connection}}\right)$

In detail, a hidden state $h_i^{l+1}$ of a node $i$ in layer $l+1$ is computed by
- (a)	Aggregating the hidden states $h_j$ of the neighbouring nodes $j \in N_i$ by summing up the products of each hidden state $h_j$ with
  - a learnable $d_l \times d_{l+1}$ parameter matrix $W_1^{(l)}$ and
  - normalization constant $c_{i,j}=\frac{1}{\sqrt{D_{i,i}D_{j,j}}}$ where $D_{i,i}$ is the node degree of node $i$. The normalization constant is particularly important, since it reduces the impact of high node degrees.
- (b)	Applying another $d_l \times d_{l+1}$ parameter matrix $W_0^{(l)}$ to the hidden state $h_i$ of node $i$ in layer $l$ (which corresponds to adding a self-loop) and adding that to the overall aggregated value 
- (c)	Applying an element-wise non-linear activation function $\sigma$ like ReLu to the overall result
- (d)	Assigning the result to node $h_i$ as its hidden state in layer $l+1$

To construct a $n$-layer GCN, these steps are repeated $n$ times for each node in parallel by using the output of the $i$-th layer as the input of the $i+1$-th layer. The original node features serve as input to the first layer, and the output of the last layer is simultaneously the output of the overall GCN. Stacking the layers in that way captures dependencies across several relational steps, i.e. after k layers, a node embedding incorporates information, both structural and feature-based, of its k-hop neighbourhood (Hamilton, 2020). These so-called vanilla GCNs are an example of *Isotropic* Graph Neural Networks, i.e. when calculating the hidden state for each node, each of its neighbors is included with equal weight (Dwivedi et al., 2020).

## Relational Graph Convolutional Networks (R-GCNs)
In this seminar, we employed an advancement of GCNs, namely Relational GCNs (R-GCNs) that was proposed by Schlichtkrull et al. (2018). R-GCNs support multi-relational data, i.e. graphs with a type assigned to each edge. A common example of such data are knowledge graphs (like DBPedia or Wikidata), which may be conceived as a collection of triples *(subject, predicate, object)*. Subject and object correspond to typed entities and the predicate denotes the relation type. A proper representation of knowledge graphs are directed, labeled multigraphs with entities as nodes and triples determining the edge relation.\
To support this kind of data, the previously presented computation of a layer is enhanced by using a separate weight matrix $W_r^{(l)}$ for each relation type $r \in R$. Broadly speaking, the hidden states of neighbouring nodes are separately transformed for each relation type and direction of an edge. The node in question is also transformed separately (corresponding to a self-loop). These transformations are then aggregated in a normalized manner and passed through an activation function. This computation is individually performed for each node in the graph. The following formula comprises the node update for a node $i$ in layer $l+1$:

> $h_i^{l+1} = \sigma\left(\sum_{r\in R}\sum_{j\in N^r_i}\frac{1}{c_{i,r}}W_r^{(l)}h_j^{(l)}+\underbrace{W_0^{(l)}h_i^{(l)}}_{\text{self-connection}}\right)$

where 
- $N^r_i$ denotes the set of neighbors of node $i$ specific to the relation $r \in R$ 
- $c_{i,r}$ is a normalization constant that can either be learned or set by hand (e.g. $c_{i,r} = |N_i^r|$).

#### Regularization
In the corresponding work, Schlichtkrull et al. (2018) further present regularization techniques in the form of basis decomposition and block-diagonal decomposition. Regularization of the weights in R-GCN layers is needed due to the increasing number of parameters with the number of relations in the graph which could lead to overfitting and large models. For that reason, Schlichtkrull et al. leveraged basis decomposition of the relation-specific weight matrices as follows:

> $W_r^{(l)} = \sum^B_{b=1}a^{(l)}_{r,b}V_b^{(l)}$

The relation-specific matrices are thus represented as linear combinations of basis transformations $V_b^{(l)} \in \mathbb{R}^{d^{(l+1)}\times d^{(l)}}$ with learnable coefficients $a^{(l)}_{r,b}$ such that only the coefficients have to be adapted for each $r \in R$ and not the complete weight matrix as previously . $B$, the number of basis functions, is a hyperparameter. The basis decomposition acts as a weight sharing mechanism between different relation types, such that the number of parameters that need to be learned is reduced. Schlichtkrull et al. further assume that basis decomposition can avert the risk of overfitting on rare relations due to the parameter sharing between rare and frequent relations. In addition to that, Schlichtkrull et al. also suggest employing block-diagonal decomposition. That means, the weight matrix $W_r^{(l)}$ is represented as a block-diagonal matrix: 

> $W_r^{(l)} = \bigoplus_{b=1}^{B}Q_{br}^{(l)}$

where $\bigoplus$ denotes the direct sum of matrices and $Q_{br}^{(l)} \in \mathbb{R}^{(d^{(l+1)}/B)\times (d^{(l)}/B)}$. Besides also reducing the number of parameters needed, this regularization mechanism forces each weight matrix $W_r^{(l)}$ to be sparse to some extent and intuitively groups related latent features (Schlichtkrull, 2018).

### Entity Classification with R-GCNs

In order to perform entity classification, softmax classifiers are applied to the output of the R-GCN, i.e., to the final embedding of each node. That way, labels are predicted. During the training of the model, the cross-entropy loss $L$ is minimized on the set of *labelled* nodes as follows:
> $L = -\sum_{i\in Y}\sum^K_{k=1}t_{i,k}\ln h_{i,k}^{(l)}$

where:
- $Y$ is the set of node indices with labels
- $K$ is the number of classes
- $t_{i,k}$ is the ground-truth label
- $h_{i,k}^{(l)}$ is the $k$-th entry of network ouput for $i$-th labeled node

In practice, the training is realized by use of (full-batch) gradient descent (Schlichtkrull, 2018).


## Implementation
Next, we will walk you through our implementation which encompasses the preparatory setup as well as the implementation of the R-GCN.
Github repository: https://github.com/phucdev/NDinDL

### Prepare the colab environment
We need to check the python and cuda version in the colab environment and adjust the installation of the requirements accordingly.

In [ ]:
!python --version

Python 3.7.10


In [ ]:
!nvidia-smi

Wed Apr 14 21:14:16 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Installing requirements

In [ ]:
# Install required packages.
!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio==0.8.1 -f https://download.pytorch.org/whl/torch_stable.html
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.8.0+cu111.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.8.0+cu111.html
!pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-1.8.0+cu111.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.8.0+cu111.html
!pip install torch-geometric

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████▌                  | 834.1MB 1.3MB/s eta 0:14:18tcmalloc: large alloc 1147494400 bytes == 0x55dc67ce2000 @  0x7f68d0f0d615 0x55dc2e3bb06c 0x55dc2e49aeba 0x55dc2e3bde8d 0x55dc2e4af99d 0x55dc2e431fe9 0x55dc2e42cb0e 0x55dc2e3bf77a 0x55dc2e431e50 0x55dc2e42cb0e 0x55dc2e3bf77a 0x55dc2e42e86a 0x55dc2e4b07c6 0x55dc2e42dee2 0x55dc2e4b07c6 0x55dc2e42dee2 0x55dc2e4b07c6 0x55dc2e42dee2 0x55dc2e4b07c6 0x55dc2e532431 0x55dc2e493049 0x55dc2e3fdc84 0x55dc2e3be8e9 0x55dc2e432ade 0x55dc2e3bf69a 0x55dc2e42da45 0x55dc2e42ce0d 0x55dc2e3bf77a 0x55dc2e42da45 0x55dc2e3bf69a 0x55dc2e42da45
     |█████████████████               | 1055.7MB 1.2MB/s eta 0:12:26tcmalloc: large alloc 1434370048 bytes == 0x55dcac338000 @  0x7f68d0f0d615 0x55dc2e3bb06c 0x55dc2e49aeba 0x55dc2e3bde8d 0x55dc2e4af99d 0x55dc2e431fe9 0x55dc2e42cb0e 0x55dc2e3bf77a 0x55dc2e431e50 0x55dc2e42cb0e 0x55dc2e3bf77a 0x55dc2e42e86a 0x55dc2e4b07c6 0x55dc2e42dee2 0

### Imports
We mainly use pytorch geometric to load the datasets, numpy and scipy to process the data and pytorch for the model.

In [ ]:
from torch_geometric.datasets import Entities
from torch_geometric.utils.num_nodes import maybe_num_nodes
import numpy as np
import scipy.sparse as sp
from scipy import stats
import torch
import torch.nn as nn
import torch.nn.functional as F
from functools import partial
import os.path as osp

### R-GCN layer
This part is inspired by the keras implementation from Thomas Kipf and pytorch based implementations:
- https://github.com/tkipf/pygcn
- https://github.com/masakicktashiro/rgcn_pytorch_implementation
- https://github.com/mjDelta/relation-gcn-pytorch

During the initialization of an R-GCN layer, everything is set up for basis decomposition. To achieve quicker convergence and higher accuracy, we employ Xavier initialization to initialize the trainable parameters (Dellinger, 2019). We use the recommmended gain value for the given non-linearity function (here: ReLu), which is an optional scaling factor. We also give the user the opportunity to define a dropout probability, although we do not use it in the experiments. Dropout is yet another regularization technique that essentially refers to randomly dropping units during the training of a neural network (Srivastava, 2014). That way, overfitting can be mitigated.

In [ ]:
class RGCNConv(nn.Module):
    def __init__(self,
                 input_dim,
                 output_dim,
                 num_rels,
                 num_bases=-1,
                 bias=False,
                 activation=None,
                 dropout=0.5,
                 is_output_layer=False):
        r"""The relational graph convolutional operator from the `"Modeling
        Relational Data with Graph Convolutional Networks"
        <https://arxiv.org/abs/1703.06103>`_ paper

        Propagation model:
        (1) $h_i^{l+1} = \sigma\left(\sum_{r\in R}\sum_{j\in N^r_i}\frac{1}{c_{i,r}}W_r^{(l)}h_j^{(l)}+
        \underbrace{W_0^{(l)}h_i^{(l)}}_{\text{self-connection}}\right)$

        where
        - $N^r_i$ denotes the set of neighbor indices of node $i$ under relation $r \in R$,
        - $c_{i,r}$ is a problem-specific normalization constant that can either be learned or chosen in advance
          (such as $c_{i,r} = |N_i^r|$).

        Neural network layer update: evaluate message passing update in parallel for every node $i \in V$.

        Parameter sharing for highly- multi-relational data: basis decomposition of relation-specific weight matrices
        (2) $W_r^{(l)} = \sum^B_{b=1}a^{(l)}_{r,b}V_b^{(l)}$

        Linear combination of basis transformations $V_b^{(l)} \in \mathbb{R}^{d^{(l+1)}\times d^{(l)}}$ with learnable
        coefficients $a^{(l)}_{r,b}$ such that only the coefficients depend on $r$. $B$, the number of basis functions,
        is a hyperparameter.

        :param input_dim: Input dimension
        :param output_dim: Output dimension
        :param num_rels: Number of relation types
        :param num_bases: Number of bases used in basis decomposition of relation-specific weight matrices
        :param bias: Optional additive bias
        :param activation: Activation function
        :param dropout: Dropout
        :param is_output_layer: Indicates whether this layer is the output layer
        """
        super(RGCNConv, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.num_rels = num_rels
        self.num_bases = num_bases
        self.bias = bias
        self.activation = activation
        self.dropout = dropout
        self.is_output_layer = is_output_layer

        # Number of bases for the basis decomposition can be less or equal to 
        # the number of relation types
        if self.num_bases <= 0 or self.num_bases > self.num_rels:
            self.num_bases = self.num_rels

        # Weight bases in equation (2)
        # V_b if self.num_bases < self.num_rels, 
        # W_r if self.num_bases == self.num_rels
        self.weight = nn.Parameter(
            torch.Tensor(self.num_bases * self.input_dim, self.output_dim))

        # Use basis decomposition otherwise if num_bases = num_rels we can just 
        # use one weight matrix per relation type
        if self.num_bases < self.num_rels:
            # linear combination coefficients a^{(l)}_{r, b} in equation (2)
            self.w_comp = nn.Parameter(
                torch.Tensor(self.num_rels, self.num_bases))

        if self.bias:
            self.b = nn.Parameter(torch.Tensor(self.output_dim))
        self.reset_parameters()

    def reset_parameters(self):
        # Initialize trainable parameters, see following link for explanation:
        # https://towardsdatascience.com/weight-initialization-in-neural-networks-a-journey-from-the-basics-to-kaiming-954fb9b47c79
        # Xavier initialization: improved weight initialization method enabling 
        # quicker convergence and higher accuracy
        # gain is an optional scaling factor, here we use the recommended gain 
        # value for the given nonlinearity function
        nn.init.xavier_uniform_(self.weight, gain=nn.init.calculate_gain('relu'))
        if self.num_bases < self.num_rels:
            nn.init.xavier_uniform_(self.w_comp, gain=nn.init.calculate_gain('relu'))
        if self.bias:
            nn.init.xavier_uniform_(self.b, gain=nn.init.calculate_gain('relu'))

    def forward(self, x, adj_t):
        supports = []   # supports holds hidden states of the nodes
        num_nodes = adj_t[0].shape[0]
        for i, adj in enumerate(adj_t):
            # Matrix multiplication of relation type specific adjacency matrices
            # with x, the node embeddings or hidden states of the nodes from the previous layer
            if x is not None:
                supports.append(torch.spmm(adj, x))
            else:
                supports.append(adj)
        # List of (num_nodes, num_nodes) tensors -> (num_nodes, num_nodes*num_rel)
        supports = torch.cat(supports, dim=1)

        # Calculate relation specific weight matrices
        if self.num_bases < self.num_rels:
            # Generate all weights from bases as in equation (2)
            weight = self.weight.reshape(self.num_bases, self.input_dim, self.output_dim).permute(1, 0, 2)

            # Matrix product: learnable coefficients a_{r, b} and basis transformations V_b
            # (self.num_rels, self.num_bases) x (self.input_dim, self.num_bases, self.output_dim)
            weight = torch.matmul(self.w_comp, weight)  # (self.input_dim, self.num_rels, self.output_dim)
            weight = weight.reshape(self.input_dim * self.num_rels, self.output_dim)
        else:
            weight = self.weight

        out = torch.spmm(supports, weight)  # (num_nodes, num_rels)

        # If x is None add dropout to output, by elementwise multiplying with 
        # column vector of ones, with dropout applied to the vector of ones.
        if x is None:
            temp = torch.ones(num_nodes).to(out.device)
            temp_drop = F.dropout(temp, self.dropout)
            out = (out.transpose(1, 0) * temp_drop).transpose(1, 0)

        if self.bias:
            out += self.b

        out = self.activation(out)
        return out

The `forward` function is basically the place where we define how the output of a layer is computed. First, we compute the relation-specific matrices based on the bases if necessary. Then we compute the product of the tensor representing the relation-specific, normalized adjacency matrices (which may also incorporate the features if any are given) and the weight matrices. Lastly, we also factor the dropout and the bias in, before applying the activation function.

### R-GCN model
This part is somewhat inspired by: https://docs.dgl.ai/en/0.4.x/tutorials/models/1_gnn/4_rgcn.html.
We borrowed the idea of building the model using separate build functions for the input, hidden and output layer in order to allow building models with multiple hidden layers. That way, we can easily stack the layers (implemented as `RGCNConv`) as it was described in the introductory section.

In [ ]:
class RGCN(torch.nn.Module):
    def __init__(self, num_nodes, h_dim, out_dim, num_rels,
                 num_bases=-1, num_hidden_layers=1, dropout=0.5, bias=False):
        """
        Implementation of R-GCN from the `"Modeling
        Relational Data with Graph Convolutional Networks"
        <https://arxiv.org/abs/1703.06103>`_ paper

        :param num_nodes: Number of nodes (input dimension)
        :param h_dim: Hidden dimension
        :param out_dim: Output dimension
        :param num_rels: Number of relation types
        :param num_bases: Number of basis functions
        :param num_hidden_layers: Number of hidden layers
        :param dropout: Dropout probability
        :param bias: Whether to use an additive bias
        """
        super(RGCN, self).__init__()
        self.num_nodes = num_nodes
        self.h_dim = h_dim
        self.out_dim = out_dim
        self.num_rels = num_rels
        self.num_bases = num_bases
        self.num_hidden_layers = num_hidden_layers
        self.dropout = dropout
        self.bias = bias

        self.layers = nn.ModuleList()
        # create rgcn layers
        self.build_model()

    def build_model(self):
        # input to hidden
        i2h = self.build_input_layer()
        self.layers.append(i2h)
        # hidden to hidden
        for _ in range(self.num_hidden_layers):
            h2h = self.build_hidden_layer()
            self.layers.append(h2h)
        # hidden to output
        h2o = self.build_output_layer()
        self.layers.append(h2o)

    def build_input_layer(self):
        return RGCNConv(self.num_nodes, self.h_dim, self.num_rels, self.num_bases, activation=F.relu,
                        dropout=self.dropout, bias=self.bias)

    def build_hidden_layer(self):
        return RGCNConv(self.h_dim, self.h_dim, self.num_rels, self.num_bases, activation=F.relu,
                        dropout=self.dropout, bias=self.bias)

    def build_output_layer(self):
        return RGCNConv(self.h_dim, self.out_dim, self.num_rels, self.num_bases, activation=partial(F.softmax, dim=-1),
                        dropout=self.dropout, is_output_layer=True, bias=self.bias)

    def reset_parameters(self):
        for layer in self.layers:
            layer.reset_parameters()

    def forward(self, x, adj_t):
        out = x
        for layer in self.layers:
            out = layer(out, adj_t)
            if not layer.is_output_layer:
                out = F.dropout(out, self.dropout, self.training)
        return out

### Training and Evaluation Functions

The training step consists of calculating the loss for the labelled (training) nodes and propagating it backwards through the network. Lastly, the weights are adapted in `optimizer.step()`. The `test` function enables us to calculate the accuracy on the test as well as the training data.

In [ ]:
def train(model, x, adj_t, optimizer, loss_fn, train_idx, train_y):
    model.train()

    # Zero grad the optimizer
    optimizer.zero_grad()
    # Feed the data into the model
    out = model(x, adj_t)
    # Feed the sliced output and label to loss_fn
    labels = torch.LongTensor(train_y).to(out.device)
    loss = loss_fn(out[train_idx], labels)

    # Backpropagation, optimizer
    loss.backward()
    optimizer.step()
    return loss.item()


@torch.no_grad()
def test(model, x, adj_t, train_idx, train_y, test_idx, test_y):
    model.eval()

    # Output of model on all data
    out = model(x, adj_t)
    # Get predicted class labels
    pred = out.argmax(dim=-1).cpu()

    # Evaluate prediction accuracy
    train_acc = pred[train_idx].eq(train_y).to(torch.float).mean()
    test_acc = pred[test_idx].eq(test_y).to(torch.float).mean()
    return train_acc.item(), test_acc.item()

### Data preprocessing functions
In order to use the data from `torch.geometric.datasets.Entities` with our R-GCN implementation, we have to convert the dataset and construct the adjacency matrices from the edge index and edge type arrays. We therefore compute a collection of adjacency matrices, with one adjacency matrix per relation type. 

In [ ]:
def get_adjacency_matrices(data):
    """
    Converts torch_geometric.datasets.entities data to relation type specific 
    adjacency matrices
    :param data: torch_geometric.datasets.entities data
    :return:
        A: list of relation type specific adjacency matrices
    """
    num_rels = data.num_rels
    num_nodes = data.num_nodes

    A = []
    source_nodes = data.edge_index[0].numpy()
    target_nodes = data.edge_index[1].numpy()

    # Get edges for given (relation) edge type and construct adjacency matrix
    for i in range(num_rels):
        indices = np.argwhere(np.asarray(data.edge_type) == i).squeeze(axis=1)
        r_source_nodes = source_nodes[indices]
        r_target_nodes = target_nodes[indices]
        a = sp.csr_matrix(
            (np.ones(len(indices)), (r_source_nodes, r_target_nodes)), 
            shape=(num_nodes, num_nodes))
        A.append(a)

    return A

The following functions serve for normalizing the matrices individually and converting them to sparse tensors. The normalization corresponds to the normalization that is performed in a R-GCN layer in order to mitigate the impact of high node degress. In this case, we use the node degree only, as it is suggested in the paper by Schlichtkrull et al. (2018). \\
We further use sparse tensors because they are memory-efficient. Sparse arrays such as our adjacency matrices, where ones indicate edges, contain a lot of elements equal to zero. Thus, we achieve a more efficient use of processor resources as well as memory if we only store and process the non-zero elements (Torch Contributors, 2019).

In [ ]:
def normalize(adj_matrix):
    """
    Normalizes the adjacency matrix
    :param adj_matrix: Adjacency matrix
    :return:
        out: Normalized adjacency matrix
    """
    node_degrees = np.array(adj_matrix.sum(axis=1)).flatten()
    # Essentially 1. / node_degrees, while avoiding division by zero warning
    norm_const = np.divide(np.ones_like(node_degrees), node_degrees, out=np.zeros_like(node_degrees),
                           where=node_degrees != 0)
    D_inv = sp.diags(norm_const)
    out = D_inv.dot(adj_matrix).tocsr()
    return out

In [ ]:
def to_sparse_tensor(sparse_array):
    """
    Converts sparse array (normalized adjacency matrix) to sparse tensor
    :param sparse_array: Sparse array (normalized adjacency matrix)
    :return:
        sparse_tensor: Converted sparse tensor
    """
    if len(sp.find(sparse_array)[-1]) > 0:
        # Get indices and values of nonzero elements in matrix
        v = torch.FloatTensor(sp.find(sparse_array)[-1])
        i = torch.LongTensor(sparse_array.nonzero())
        shape = sparse_array.shape
        sparse_tensor = torch.sparse_coo_tensor(i, v, torch.Size(shape))
    else:
        sparse_tensor = torch.sparse_coo_tensor(sparse_array.shape[0], sparse_array.shape[1])
    return sparse_tensor

## Experiments
In this section, we investigate the accuracy of the model that was implemented within the context of this seminar project. For this purpose, we relied on the same datasets that Schlichtkrull et al. (2018) used in their experiments. In particular, they used the datasets *AM*, *BGS*, *AIFB* and *MUTAG*. These datasets are part of a benchmark for evaluating machine learning approaches in the context of the semantic web (Ristoski, 2016). Relations in those datasets do not necessarily represent the triples of the knowledge graph only but may also provide information on the presence or absence of a feature. The node classification task consists in predicting properties for a specific group of entities for each dataset (Schlichtkrull, 2018). The table below shows the key figures of each dataset in detail. 

### Description of the Datasets
The *AIFB* dataset describes the organizational structure of the AIFB, a research institute at the university *Karlsruher Institut für Technologie*. The dataset includes
178 entities representing people that are assigned to a research group within the AIFB. A possible classification task consists in predicting the research group for each entity. In a pre-processing step, the smallest group was omitted as well as the *employs* relation due to it being the inverse of the *affiliate* relation (Ristoski, 2016). In addition to that, the dataset contains information on research papers, external people or projects (Bloehdorn, 2007).

The *MUTAG* dataset comprises data on complex, potentially carcinogenic molecules. In this case, the property *isMutagenic* serves as a label (Ristoski, 2016). The dataset represents a set of  molecular graphs in RDF format, where relations encode atomic bonds or the presence of a certain feature (Schlichtkrull, 2018). It is generally used for graph classification (Papers with Code, n.d.), but in this case it has been prepared in a way such that the carcinogenicity is predicted for each compound in a node classification setting (DL-Learner, n.d.; "MUTAG/readme.txt", 2015).

The *AM* dataset describes artifacts maintained by the Amsterdam Museum, where each artifact refers to metadata on its production, material and content. Each artifact may also be linked to other artifacts and is further labelled with an artifact category, which is used for classification in this case. Ristoski et al. (2016) drew and pre-processed a stratitified random sample of 1000 instances from the complete dataset before including it in the benchmark in its current form. The pre-processing consisted in omitting a relation due to its high correlation with the artifact category. 

The *BGS* dataset contains geological measurements on so-called named rock units that were collected by the British Geological Survey. Various properties are assigned to these named rock units that describe rock formations (see https://data.bgs.ac.uk/id/Lexicon/NamedRockUnit/MTON for an example of such a named rock unit). In this case, the so-called lithogenetic type, the "mode of origin" (British Geological Survey, 2021) of rock formations, is used to divide the rock units into different classes. The rock units belonging to the two largest lithogenesis classes, namely "GLACI" (glacial) and "FLUV" (fluviatile) ("Lithogenesis types", 2013; "BGS/readme.txt", 2015), form the *BGS* dataset as it was used by de Vries et al. (2013) and then adopted by Ristoski et al. (2016) in their proposed benchmark. Thus, the dataset encompasses 146 named
rock units, for which the lithogenesis property can be predicted. In the RDF representation of this dataset, relations also either represent the presence of a certain feature or feature hierarchy (Schlichtkrull, 2018).

The BGS as well as the MUTAG datasets are special in the sense that high-degree nodes encoding a certain feature act as a hub by linking labeled entities to each other (Schlichtkrull, 2018). These datasets are available in Resource Description Framework (RDF) format. In addition to that, all four datasets are also directly available via `torch.geometric.datasets.Entities`. 

 \\


| Dataset | AIFB | MUTAG | BGS | AM |
| ------- | ---- | ----- | --- | ---|
|Entities | 8,285| 23,644|333,845| 1,666,764|
|Relations| 45 | 23 | 103 | 133 |
|Edges | 29,043 | 74,227 | 916,199 | 5,988,321|
|Labeled | 176 | 340 | 146 | 1,000 |
|Classes | 4 | 2 | 2 | 11 |

<center> <i>Labeled</i> denotes those entities that carry labels. Thus, they are to be classified during the training or testing stage, respectively. <br> Source of the table: (Schlichtkrull, 2018) </center>

### Loading the Data
In theory, our model should work with all four datasets. However, BGS and AM contain huge graphs, which require lots of memory. That is why we recommend to only use AIFB or MUTAG.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

The following function is used to load the data via `pytorch geometric`. For that, we make use of the previously defined `get_adjacency_matrices` function to obtain a set of relation type specific adjacency matrices, that are then normalized and converted to sparse tensors.

In [ ]:
def load_data(dataset_name):
  # Load data via pytorch geometric
  path = osp.join('.', 'data', 'Entities')
  dataset = Entities(path, dataset_name)
  data = dataset[0]

  data.num_nodes = maybe_num_nodes(data.edge_index)
  data.num_rels = dataset.num_relations

  # Construct relation type specific adjacency matrices from data.edge_index and data.edge_type in utils
  A = get_adjacency_matrices(data)

  adj_t = []
  # Normalize matrices individually and convert to sparse tensors
  for a in A:
      nor_a = normalize(a)
      if len(nor_a.nonzero()[0]) > 0:
          tensor_a = to_sparse_tensor(nor_a)
          adj_t.append(tensor_a.to(device))

  # Replace if features are available
  x = None    
  return dataset, data, adj_t, x

### Experiment Function
For running the experiments, we first create a R-GCN with its parameters set to the given arguments. We further train and evaluate the model for the number of times that is given by `runs`. The optimizer and loss function are initialized each time anew, the parameters are reset in each run as well. We eventually report on the best test accuracy per run as well as the average accuracy with standard error throughout the runs.

In [ ]:
def run_experiment(dataset_name, args, runs=10):
  dataset, data, adj_t, x = load_data(dataset_name)
  # Initialize RGCN model
  model = RGCN(
      num_nodes=data.num_nodes,
      h_dim=args["h_dim"],
      out_dim=dataset.num_classes,
      num_rels=dataset.num_relations,
      num_bases=args["num_bases"],
      dropout=args["dropout"]
  ).to(device)

  test_accs = []

  for i in range(1, runs+1):
    print('------------------------------------------------')
    print(f'Model run {i}')
    print('------------------------------------------------')
    # Reset the parameters to initial random values
    model.reset_parameters()

    optimizer = torch.optim.Adam(model.parameters(), lr=args["lr"], weight_decay=args["l2"])
    loss_fn = nn.CrossEntropyLoss()

    best_test_acc = 0
    # Train and evaluate model
    for epoch in range(1, args["epochs"] + 1):
        loss = train(model, x, adj_t, optimizer, loss_fn, data.train_idx, data.train_y)
        train_acc, test_acc = test(model, x, adj_t, data.train_idx, data.train_y, data.test_idx, data.test_y)
        if test_acc > best_test_acc:
          best_test_acc = test_acc
        if epoch == 1 or (epoch % 10) == 0:
          print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}, Train: {train_acc:.4f} '
                f'Test: {test_acc:.4f}')
    test_accs.append(test_acc)  # alternatively use the best test acc
    print(f'Best test accuracy: {best_test_acc:.4f}')
  
  avg_test_acc = np.mean(test_accs)
  sem_test_acc = stats.sem(test_accs)

  print('------------------------------------------------')
  print(f'Average test accuracy over {runs} runs: {100 * avg_test_acc:.2f}+-{100 * sem_test_acc:.2f}')

### Training and Evaluation
In our experiments, we train a 2-layer R-GCN with 16 hidden units, i.e., the hidden node representations have a dimension of 16. We run the experiments for 50 epochs with a learning rate of 0.01, while using the Adam optimizer (Kingma, 2014). Adam is an optimization algorithm that adapts the learning rates for each parameter individually. It has been proposed for use in the context of large-scale and high-dimensional machine learning problems. 
The normalization constant $c_{i,r}$ is set to $|N_{i}^{r}|$, i.e. the average of all incoming messages from a particular relation type. (See the previous function definitions: the normalization constant is realized by splitting the adjacency matrix up into relation-specific adjacency matrices and then normalizing them using the node degrees within each matrix.) We refrained from hyperparameter tuning and adopted the experimental settings that were used by Schlichtkrull et al. (2018).

### Evaluation on AIFB 

When evaluating our model on the AIFB dataset, we achieve a test accuracy of 95.28+-0.83 after 50 epochs of training and averaged over 10 runs with standard error. In comparison to that, Schlichtkrull et al. (2018) report on an accuracy of 95.83+-0.62.

In [ ]:
# Parameters from the RGCN paper
args = {
        'h_dim': 16,
        'num_bases': -1,  # -1: no basis decomposition, use one weight matrix for each relation
        'num_hidden_layers': 0,
        'dropout': 0.,
        'lr': 0.01,
        'l2': 0.,
        'bias': False,
        'epochs': 50,
    }

In [ ]:
run_experiment(dataset_name="AIFB", args=args, runs=10)

Extracting data/Entities/aifb.tgz
Processing...
Done!
------------------------------------------------
Model run 1
------------------------------------------------
Epoch: 01, Loss: 1.3862, Train: 0.6071 Test: 0.5556
Epoch: 10, Loss: 0.9213, Train: 0.8357 Test: 0.8333
Epoch: 20, Loss: 0.8552, Train: 0.8857 Test: 0.8889
Epoch: 30, Loss: 0.8439, Train: 0.9000 Test: 0.8889
Epoch: 40, Loss: 0.7986, Train: 0.9500 Test: 0.9167
Epoch: 50, Loss: 0.7658, Train: 0.9786 Test: 0.9722
Best test accuracy: 0.9722
------------------------------------------------
Model run 2
------------------------------------------------
Epoch: 01, Loss: 1.3865, Train: 0.4643 Test: 0.4444
Epoch: 10, Loss: 0.9579, Train: 0.8143 Test: 0.7778
Epoch: 20, Loss: 0.8166, Train: 0.9429 Test: 0.9722
Epoch: 30, Loss: 0.7754, Train: 0.9786 Test: 0.9722
Epoch: 40, Loss: 0.7646, Train: 0.9786 Test: 0.9444
Epoch: 50, Loss: 0.7630, Train: 0.9786 Test: 0.9167
Best test accuracy: 0.9722
------------------------------------------------

### Evaluation on MUTAG
When evaluating our model on the MUTAG dataset, we achieve a test accuracy of 76.91+-1.16 after 50 epochs of training and averaged over 10 runs with standard error. In comparison to that, Schlichtkrull et al. (2018) report on an accuracy of 73.23+-0.48.

In [ ]:
dataset_name = "MUTAG"  # choices=['AIFB', 'MUTAG', 'BGS', 'AM']

In [ ]:
# Parameters from the RGCN paper
args = {
        'h_dim': 16,
        'num_bases': 30,
        'num_hidden_layers': 0,
        'dropout': 0.,
        'lr': 0.01,
        'l2': 0.0005,
        'bias': False,
        'epochs': 50,
    }

In [ ]:
run_experiment(dataset_name="MUTAG", args=args, runs=10)

------------------------------------------------
Model run 1
------------------------------------------------
Epoch: 01, Loss: 0.6931, Train: 0.6213 Test: 0.6912
Epoch: 10, Loss: 0.3921, Train: 0.9706 Test: 0.7941
Epoch: 20, Loss: 0.3145, Train: 1.0000 Test: 0.6912
Epoch: 30, Loss: 0.3141, Train: 1.0000 Test: 0.7647
Epoch: 40, Loss: 0.3148, Train: 0.9963 Test: 0.7647
Epoch: 50, Loss: 0.3135, Train: 1.0000 Test: 0.7941
Best test accuracy: 0.7941
------------------------------------------------
Model run 2
------------------------------------------------
Epoch: 01, Loss: 0.6928, Train: 0.6250 Test: 0.6912
Epoch: 10, Loss: 0.3742, Train: 0.9706 Test: 0.7794
Epoch: 20, Loss: 0.3161, Train: 1.0000 Test: 0.7794
Epoch: 30, Loss: 0.3148, Train: 1.0000 Test: 0.8088
Epoch: 40, Loss: 0.3146, Train: 1.0000 Test: 0.7794
Epoch: 50, Loss: 0.3149, Train: 1.0000 Test: 0.8235
Best test accuracy: 0.8235
------------------------------------------------
Model run 3
-----------------------------------------

## More Background on (R-)GCNs

### Limitations of (R-)GCNs

R-GCNs are limited with respect to several aspects (Hamilton, 2020): 
1. *Limited Support for edge features*: They do not support all kinds of edge features. To be specific, they only support discrete edge features, i.e., edge types.
2. *Over-Smoothing*: Several iterations of the GNN message passing procedure may lead to node embeddings becoming very similar to each other, when the information from neighborhood aggregation dominates the update of the node representation. This poses a problem when building deep GNNs that aim at capturing longer-term dependencies.
3. *Same normalization constants for all neighbors*: Currently, the same normalization constant is used for each neighbor during the update of a node presentation, so we cannot treat neighbors with varying significance differently.


### Extensions of (R-)GCNs
Several extensions tackle the aforementioned deficiencies. For example, attention-based approaches like Graph Attention Networks (Veličković, 2017) replace normalization constants with learned attention weights, which addresses issue (2) from the previous section. Deep GCNs (Li, 2019) address issue (3) from the previous section by using some tricks to overcome the vanishing gradient problem and thus, enable the exploitation of long-term dependencies.

### Other Application Areas of (R-)GCNs
GNNs are put to use in various application fields: for example in machine translation, where they are used together with dependency graphs to create embeddings enriched with syntactic information (Bastings, 2017). Recommender systems constitute another application area of GNNs, since user-to-item interaction graphs and social graphs can be leveraged. Item embeddings may be learned and used for item-item recommendation, in order to recommend themed collections (e.g. playlists, “feed” content) (Ying, 2018).

## Summary
In this seminar project, we presented the mechanics of Graph Convolutional Networks (GCNs) as well as their extension, Relational Graph Convolutional Networks (R-GCNs). Both build upon the same message passing mechanism: in each iteration (layer), every node aggregates information from its local neighborhood in parallel. The more iterations pass, the more information the nodes can gather from further reaches of the graph. The R-GCN introduces the relation-specific aggregation and normalization in order to deal with multi-relational data. We implemented a R-GCN and evaluated it on the datasets AIFB and MUTAG, that were also used in the original paper (Schlichtkrull, 2018). Similar results to those reported in the original paper were achieved.

## Resources

### Papers and books:

* Bastings, J., Titov, I., Aziz, W., Marcheggiani, D., & Sima'an, K. (2017). Graph convolutional encoders for syntax-aware neural machine translation. *arXiv preprint* arXiv:1704.04675.

* Bloehdorn, S., & Sure, Y. (2007). Kernel methods for mining instance data in ontologies. In *The Semantic Web* (pp. 58-71). Springer, Berlin, Heidelberg.

* de Vries, G. K. (2013, September). A fast approximation of the Weisfeiler-Lehman graph kernel for RDF data. In *Joint European Conference on Machine Learning and Knowledge Discovery in Databases* (pp. 606-621). Springer, Berlin, Heidelberg.

* Dwivedi, V. P., Joshi, C. K., Laurent, T., Bengio, Y., & Bresson, X. (2020). Benchmarking graph neural networks. *arXiv preprint* arXiv:2003.00982.

* Hamilton, W. L. (2020). Graph representation learning. *Synthesis Lectures on Artifical Intelligence and Machine Learning*, 14(3), 1-159.

* Kingma, D. P., & Ba, J. (2014). Adam: A method for stochastic optimization. *arXiv preprint* arXiv:1412.6980.

* Kipf, T. N., & Welling, M. (2016). Semi-supervised classification with graph convolutional networks. *arXiv preprint* arXiv:1609.02907.

* Kipf, T. N. (2020). Deep learning with graph-structured representations.

* LeCun, Y. (1989). Generalization and network design strategies. Technical Report
CRG-TR-89-4, University of Toronto.

* Li, G., Muller, M., Thabet, A., & Ghanem, B. (2019). Deepgcns: Can gcns go as deep as cnns?. In *Proceedings of the IEEE/CVF International Conference on Computer Vision* (pp. 9267-9276).

* Srivastava, N., Hinton, G., Krizhevsky, A., Sutskever, I., & Salakhutdinov, R. (2014). Dropout: a simple way to prevent neural networks from overfitting. *The journal of machine learning research*, 15(1), 1929-1958.

* Ristoski, P., De Vries, G. K. D., & Paulheim, H. (2016, October). A collection of benchmark datasets for systematic evaluations of machine learning on the semantic web. In *International Semantic Web Conference* (pp. 186-194). Springer, Cham.

* Rumelhart, D., Hinton, G., and Williams, R. (1986a). Learning representations by back-propagating errors. Nature, 323, 533–536.

* Schlichtkrull, M., Kipf, T. N., Bloem, P., Van Den Berg, R., Titov, I., & Welling, M. (2018, June). Modeling relational data with graph convolutional networks. In *European semantic web conference* (pp. 593-607). Springer, Cham.

* Veličković, P., Cucurull, G., Casanova, A., Romero, A., Lio, P., & Bengio, Y. (2017). Graph attention networks. *arXiv preprint* arXiv:1710.10903.

* Ying, R., He, R., Chen, K., Eksombatchai, P., Hamilton, W. L., & Leskovec, J. (2018, July). Graph convolutional neural networks for web-scale recommender systems. In *Proceedings of the 24th ACM SIGKDD International Conference on Knowledge Discovery & Data Mining* (pp. 974-983).



### Blog Posts:
- Dellinger, J. (2019, 03. April). *Weight Initialization in Neural Networks: A Journey From the Basics to Kaiming*. URL https://towardsdatascience.com/weight-initialization-in-neural-networks-a-journey-from-the-basics-to-kaiming-954fb9b47c79
- Kipf, T. (2016, 30. September). Graph Convolutional Networks. URL http://tkipf.github.io/graph-convolutional-networks/
- Skovgaard Jepsen, T. (2018, 18. September). *How to do Deep Learning on Graphs with Graph Convolutional Networks: Part 1: A High-Level Introduction to Graph Convolutional Networks*. URL https://towardsdatascience.com/how-to-do-deep-learning-on-graphs-with-graph-convolutional-networks-7d2250723780
- Skovgaard Jepsen, T. (2019, 20. January). *How to do Deep Learning on Graphs with Graph Convolutional Networks: Part 2: Semi-Supervised Learning with Spectral Graph Convolutions*. URL https://towardsdatascience.com/how-to-do-deep-learning-on-graphs-with-graph-convolutional-networks-62acf5b143d0

### Documentation
- Torch Contributors (2019). *Torch.Sparse*. URL https://pytorch.org/docs/stable/sparse.html

### Jupyter Notebook Tutorials:
- https://github.com/TobiasSkovgaardJepsen/posts/blob/master/HowToDoDeepLearningOnGraphsWithGraphConvolutionalNetworks/Part2_SemiSupervisedLearningWithSpectralGraphConvolutions/notebook.ipynb

### Keras Implementation:
- https://github.com/tkipf/relational-gcn

### PyTorch Implementations
- https://github.com/tkipf/pygcn 
- https://github.com/masakicktashiro/rgcn_pytorch_implementation
- https://github.com/mjDelta/relation-gcn-pytorch
- https://docs.dgl.ai/en/0.4.x/tutorials/models/1_gnn/4_rgcn.html
- https://github.com/rusty1s/pytorch_geometric/blob/master/examples/rgcn.py


### Datasets directly available via `torch.geometric.datasets.Entities`: AIFB, MUTAG
- Overview: https://www.uni-mannheim.de/dws/research/resources/sw4ml-benchmark/
- Download: http://data.dws.informatik.uni-mannheim.de/rmlod/LOD_ML_Datasets/data/datasets/RDF_Datasets/

## Information on Datasets
- British Geological Survey (n.d.). *BGS Lexicon of Named Rock Units*. URL https://www.bgs.ac.uk/technologies/the-bgs-lexicon-of-named-rock-units/ (last access: 23.05.2021)
- British Geological Survey (2020). *The BGS Lexicon of Named Rock Units [Linked Open Data]*. Keyworth, Nottingham. Available from http://data.bgs.ac.uk/ref/Lexicon (last access: 23.05.2021)
- British Geological Society (2021). *BGS Geology - 50k (DiGMapGB-50) Superficial version 8.* URL https://data.gov.uk/dataset/f2f96865-4ad5-4adc-b0f6-3e79e71f10c0/bgs-geology-50k-digmapgb-50-superficial-version-8 (last access: 23.05.2021)
- Lithogenesis Types [Data File] (2013). URL http://data.dws.informatik.uni-mannheim.de/rmlod/LOD_ML_Datasets/data/datasets/RDF_Datasets/BGS/data_bgs/Lexicon_LithogeneticType.nt (last access: 23.05.2021)
- BGS/readme.txt [Data File] (2015). URL http://data.dws.informatik.uni-mannheim.de/rmlod/LOD_ML_Datasets/data/datasets/RDF_Datasets/BGS/readme.txt
- Papers with Code (n.d.). MUTAG. URL https://paperswithcode.com/dataset/mutag (last access: 23.05.2021)
- DL-Learner (n.d.). Carcinogenesis Predictions using DL-Learner. URL http://dl-learner.org/community/carcinogenesis (last access: 23.05.2021)
- MUTAG/readme.txt [Data File] (2015). URL http://data.dws.informatik.uni-mannheim.de/rmlod/LOD_ML_Datasets/data/datasets/RDF_Datasets/MUTAG/README.txt (last access: 23.05.2021)
- MUTAG/carcinogenesis.owl [Data File] (2015). URL http://data.dws.informatik.uni-mannheim.de/rmlod/LOD_ML_Datasets/data/datasets/RDF_Datasets/MUTAG/carcinogenesis.owl (last access: 23.05.2021)
- MUTAG/completeDataset.tsv [Data File] (2015). URL http://data.dws.informatik.uni-mannheim.de/rmlod/LOD_ML_Datasets/data/datasets/RDF_Datasets/MUTAG/completeDataset.tsv (last access: 23.05.2021)